# Selects concrete columns with a certain amount of capacity ratio

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import comtypes.client

import json
import tkinter as tk
from tkinter import filedialog
import lib.csiapi as csiapi
import lib.util as util

%load_ext autoreload
%autoreload 2

In [2]:
# Initialization of ETABSObject
"""
ETABS_path_exe = r'C:/Program Files/Computers and Structures/ETABS 21_2/ETABS.exe'
if not os.path.isfile(ETABS_path_exe):
    root = tk.Tk()
    root.withdraw()
    ETABS_path = filedialog.askopenfilename(parent=root)
    root.destroy()
"""
# create API helper object
helper = comtypes.client.CreateObject("ETABSv1.Helper")
helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)
print("Succesfully created helper object")

try:
    #get the active ETABS object
    ETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject") 
    SapModel = ETABSObject.SapModel
    print("Succesfully created SapModel")
except (OSError, comtypes.COMError):  
    print("No running instance of the program found or failed to attach.")
    sys.exit(-1)

# create folder for temporary csv files
csv_folder_path = os.getcwd() + r'\csv'
if not os.path.exists(csv_folder_path):
    os.makedirs(csv_folder_path)

Succesfully created helper object
Succesfully created SapModel


In [3]:
SapModel.Analyze.RunAnalysis()
SapModel.DesignConcrete.StartDesign()

0

In [458]:
cantilever_prefix = 'CL'
beam_prefix = 'B'
girder_prefix = 'G'
girder_reinforcement = 22
cantilever_reinforcement = 19
beam_reinforcement = 19

concrete_design_code = SapModel.DesignConcrete.GetCode()
raw_beam_reinforcement_results = csiapi.get_table_display(SapModel, 'Concrete Beam Design Summary - ' + concrete_design_code[0])

In [460]:
# Assinging locations to beams

beam_unique_name = raw_beam_reinforcement_results['UniqueName']
beam_station = raw_beam_reinforcement_results['Station']
beam_design_sect = raw_beam_reinforcement_results['DesignSect']
beam_as_top = raw_beam_reinforcement_results['AsTopTotal'] # for cantilever checking
max_station = []
min_station = []
ln = []
reinforcement_location = []
for idx in range(len(beam_unique_name)):
# for i in range(1):
    i = beam_unique_name[idx]
    local_beam_station = beam_station[idx]
    local_design_sect = beam_design_sect[idx]
    local_beam_stations = (beam_station[i == beam_unique_name]).to_numpy()
    # max_station[i] = max(local_beam_stations)
    # min_station[i] = min(local_beam_stations)
    local_max_station = max(local_beam_stations)
    local_min_station = min(local_beam_stations)
    local_ln = local_max_station - local_min_station
    max_station.append(local_max_station)
    min_station.append(local_min_station)
    # max_station.concatenate(local_max_station)
    # min_station.concatenate(local_min_station)
    ln.append(local_max_station-local_min_station)
    if cantilever_prefix in local_design_sect:
        local_beam_as_top = (beam_as_top[i == beam_unique_name]).to_numpy()
        condition1 = local_beam_as_top[0] > local_beam_as_top[-1]
        condition2 = local_beam_station <= (local_ln / 2 + local_min_station)
        # if condition1 & condition2:
        #    local_location = 'End'
        if ~(condition1 ^ condition2):
            local_location = 'End'
        else:
            local_location = 'Mid'
    elif (girder_prefix in local_design_sect) | (beam_prefix in local_design_sect):
        if local_beam_station < (1/4*local_ln + local_min_station):
            local_location = 'End-I'
        elif local_beam_station > (3/4*local_ln + local_min_station):
            local_location = 'End-J'
        else:
            local_location = 'Mid'
    reinforcement_location.append(local_location)
    # ln.concatenate(local_max_station-local_min_station)
"""
for i in beam_unique_name.unique():
    local_reinforcement_location_i = local_beam_stations < (1/4*local_ln + local_min_station)
    local_reinforcement_location_j = local_beam_stations > (3/4*local_ln + local_min_station)  
    local_reinforcement_location_m = (~(local_reinforcement_location_i | local_reinforcement_location_j))
    # local_reinforcement = local_reinforcement_location_i * 1 + local_reinforcement_location_m * 2 + local_reinforcement_location_j * 3
    # reinforcement_location.append(local_reinforcement)
    reinforcement_location = np.append(reinforcement_location, local_reinforcement)
"""
raw_beam_reinforcement_results['Location'] = reinforcement_location

In [461]:
util.open_as_csv(raw_beam_reinforcement_results)

In [552]:
# Joining tables

beam_unique_name_table = pd.DataFrame(raw_beam_reinforcement_results['UniqueName'].unique(), columns = ['UniqueName'])
beam_assignments = csiapi.get_table_display(SapModel, 'Frame Assignments - Summary')
beam_rectangular_sections = csiapi.get_table_display(SapModel, 'Frame Section Property Definitions - Concrete Rectangular')
concrete_materials = csiapi.get_table_display(SapModel, 'Material Properties - Concrete Data')

beam_assignments = beam_assignments[['UniqueName', 'Design Section']]
beam_rectangular_sections = beam_rectangular_sections[['Name', 'Material', 'Depth', 'Width']]
concrete_materials = concrete_materials[['Material', 'Fc']] # Fc' in kN/m^2

beam_rectangular_sections = pd.merge(beam_rectangular_sections, concrete_materials, how = 'inner')
beam_unique_name_table = pd.merge(beam_unique_name_table, beam_assignments, how = 'inner', on = 'UniqueName')
beam_unique_name_table = pd.merge(beam_unique_name_table, beam_rectangular_sections, how = 'inner', left_on = 'Design Section', right_on = 'Name', copy = False)
beam_unique_name_table.drop(columns = ['Name'], inplace = True)

# simplify both end-I and end-J to end
as_top_end = []
as_bot_end = []
as_top_mid = []
as_bot_mid = []
# assume cover of 60 mm
effective_area = (beam_unique_name_table['Depth'] - 0.06) * beam_unique_name_table['Width']
as_min_coefficient = pd.DataFrame(0.25*(beam_unique_name_table['Fc']/1000)**0.5)
as_min_coefficient['coeff2'] = 1.4
as_min_coefficient = as_min_coefficient.max(axis=1)
as_min = as_min_coefficient * effective_area

for i in beam_unique_name_table['UniqueName']:
    local_beam_table = raw_beam_reinforcement_results[i == raw_beam_reinforcement_results['UniqueName']]
    local_as_top_end = (local_beam_table['AsTopTotal'][local_beam_table['Location'].str.contains('End')]).max()
    local_as_bot_end = (local_beam_table['AsBotTotal'][local_beam_table['Location'].str.contains('End')]).max()
    local_as_top_mid = (local_beam_table['AsTopTotal'][local_beam_table['Location'].str.contains('Mid')]).max()
    local_as_bot_mid = (local_beam_table['AsBotTotal'][local_beam_table['Location'].str.contains('Mid')]).max()
    as_top_end.append(local_as_top_end)
    as_bot_end.append(local_as_bot_end)
    as_top_mid.append(local_as_top_mid)
    as_bot_mid.append(local_as_bot_mid)
    # no_bars    
    pass
beam_unique_name_table['AsTopEnd'] = as_top_end
beam_unique_name_table['AsBotEnd'] = as_bot_end
beam_unique_name_table['AsTopMid'] = as_top_mid
beam_unique_name_table['AsBotMid'] = as_bot_mid
beam_unique_name_table['AsMin'] = as_min

In [554]:
as_min

0      0.1848
1      0.1848
2      0.1848
3      0.1848
4      0.1848
        ...  
494    0.2268
495    0.2268
496    0.2268
497    0.2268
498    0.2268
Length: 499, dtype: float64

In [558]:
pd.to_csv(as_min)

AttributeError: module 'pandas' has no attribute 'to_csv'